In [ ]:
%cd /content/drive/MyDrive/research/PROJ201 20221/Detect English Text

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data/test/Politics/International politics_videos.csv')
data

In [ ]:
!pip install google-api-python-client

In [ ]:
import googleapiclient.discovery

In [ ]:
def parse_JSON(response):
    df_videos = pd.json_normalize(response['items'])
    return df_videos

In [ ]:
def extract_youtube_info(id):
  api_service_name = "youtube"
  api_version = "v3"
  my_API_Key = ""
  youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = my_API_Key)
  request = youtube.videos().list(part="snippet",
                                  id=id,
                                  )
  response = request.execute()
  return parse_JSON(response)

In [ ]:
id = 'uczMaul5Kxs'
tmp = extract_youtube_info(id)
tmp

In [ ]:
tmp.loc[0, 'snippet.title']

In [ ]:
tmp.loc[0, 'snippet.description']

In [ ]:
!pip install langid

In [ ]:
import langid
def langid_predict(txt):
    pred = langid.classify(txt)[0]
    if pred=='en':
      return 'en'
    else:
      return 'non-en'

In [ ]:
langid_predict(tmp.loc[0, 'snippet.title'])

In [ ]:
langid_predict(tmp.loc[0, 'snippet.description'])

In [ ]:
import string
def remove_punctuations(text):
  new_text = ''
  for ch in text:
    if not (ch in string.punctuation):
      new_text += ch
  return new_text

In [ ]:
import re
def clean_tweets(tweet):
  # Remove Emoji
  # tweet = remove_emojis(tweet)
  # Remove tags
  tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
  # Remove hashtags
  tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
  # Remove links
  tweet = re.sub(r'http\S+', '', tweet)
  # Remove new line
  tweet = re.sub('\n+', ' ', tweet)
  # tweet = re.sub('\n', ' ', tweet)
  # Remove emails
  tweet = re.sub(r'[a-zA-Z\d#!%\$‘&\+\*–/=\?\^_`\.\{\|\}~]+@[a-zA-Z\d]+\.[a-zA-Z\.]+', "", tweet)
  # Remove punctuation
  tweet = remove_punctuations(tweet)
  # Remove Repeated spaces
  tweet = re.sub(' +', ' ', tweet)
  tweet = tweet.strip()
  return tweet

In [ ]:
def pred_title_desc_lang(id):
  video_info = extract_youtube_info(id)
  video_info = video_info[['snippet.title', 'snippet.description']]
  title = clean_tweets(video_info.loc[0, 'snippet.title'])
  title_lang = langid_predict(title)
  desc = clean_tweets(video_info.loc[0, 'snippet.description'])
  desc_lang = langid_predict(desc)
  return pd.Series([title_lang, desc_lang])

In [ ]:
pred_title_desc_lang(id)

In [ ]:
data[['title_lang', 'desc_lang']] = data.apply(lambda row: pred_title_desc_lang(row['id.videoId']), axis=1)
data

In [ ]:
import glob
for file_path in glob.glob('data/test/*/*.csv'):
  data = pd.read_csv(file_path)
  if 'title_lang' in data.columns:
    continue
  data[['title_lang', 'desc_lang']] = data.apply(lambda row: pred_title_desc_lang(row['id.videoId']), axis=1)
  data.to_csv(file_path, index=False)
  print('Finish:', file_path)
  # break